###  importing relevant libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from random import random
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix
#time.sleep(3600)

### Discovering Labels

In [3]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    df["Label"]=df["Label"].replace(renamer)
    df=df.dropna()
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [4]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [5]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [6]:
find_the_way("small",".csv")

['small\\uk-vpn.csv', 'small\\uk.csv', 'small\\us-vpn.csv', 'small\\us.csv']

In [7]:
renamer={}

In [8]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['eth.src']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [9]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):

    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [10]:


def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]


In [11]:
def MLCV(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        fold=5
        repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):


            rnd = random()
            
            kfold = sklearn.model_selection.KFold(n_splits=fold, shuffle=True, random_state=int(rnd*100))  
            cv=0
            df = pd.read_csv(loop1,usecols=cols)#,header=None )
            ##df = df.reset_index(drop=True)
            df=df.fillna(0)
            macs=df["eth.src"]
            del df["eth.src"]
            #del df["MAC"] # if dataset has MAC colomn please uncomment this line
            X =df[df.columns[0:-1]]
            X=np.array(X)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y=df[df.columns[-1]].cat.codes  
            X.shape

            for train_index, test_index in kfold.split(X):
                #try:
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index] 
                m_train, m_test = macs[train_index], macs[test_index] 

            
            

                results_y=[]
                cv+=1
                results_y.append(y_test)
    
    
                #machine learning algorithm is applied in this section
                clf = ml_list[ii]#choose algorithm from ml_list dictionary
                second=time.time()
                clf.fit(X_train, y_train)
                train_time=(float((time.time()-second)) )
                second=time.time()
                predict =clf.predict(X_test)
                test_time=(float((time.time()-second)) )
                if step==1:
                    altime=0
                    line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                else:
                    predict,altime=merged(m_test,predict,step,mixed)
                    line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                    
                    
                lines.append(line)
                df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
                if cm.empty:
                    cm =df_cm
                else:
                    cm = cm.add(df_cm, fill_value=0)
                
        class_based_results=class_based_results/(repetition*fold)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()

        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//(repetition*fold)
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



In [13]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=25
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            df=df.dropna()

            m_train=df["eth.src"]
            del df["eth.src"]
            
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df=df.dropna()

            m_test=df["eth.src"]
            del df["eth.src"]
            
            df["Label"]=df["Label"].replace(renamer)
            #df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes


            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



# Machine learning applications 

# Aalto Dataset

In [14]:
feature= ['dstport',
  'dstport_class',
  'http.chat',
  'http.notification',
  'http.request.method',
  'ip.flags.df',
  'ip.len',
  'ip.proto',
  'ip.ttl',
  'srcport',
  'srcport_class',
  'tcp.ack',
  'tcp.analysis.ack_rtt',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.initial_rtt',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.flags.syn',
  'tcp.hdr_len',
  'tcp.len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.time_relative',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'tcp.window_size_value',
  'tls.record.length',
  'udp.checksum.status',
  'udp.dstport',
  'udp.srcport',
  'udp.time_delta',
  'udp.time_relative',"eth.src","Label"]

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost

# Cross-Validation

In [16]:

ml_list={#"DT":DecisionTreeClassifier()}
    "RF":RandomForestClassifier(bootstrap=False,criterion="entropy",max_depth=21,max_features=9,min_samples_split=7,n_estimators=168)}

In [17]:
file_list={"UKVPN":'small/uk-vpn.csv',
           "UK":'small/uk.csv',
           "USVPN":'small/us-vpn.csv',
           "US":'small/us.csv'}

### Normal

In [31]:
savehere="results/cv/1/"
folder(savehere)
step=1
mixed=0
for file in file_list:
    print(file)
    train=file_list[file]
    test=file_list[file]
    output_csv=f"{savehere}{file}_ML_.csv"
    target_names=target_name(file_list[file])
    MLCV(train,test,output_csv,feature,step,mixed,file)   


UKVPN
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN        0     1  RF    0.984    0.961   0.97   0.961  0.965  0.979      0  174.707     1.563          0
 1  UKVPN        0     2  RF    0.986    0.966   0.974  0.966  0.97   0.981      0  169.86      1.67           0
 2  UKVPN        0     3  RF    0.984    0.962   0.961  0.962  0.961  0.979      0  173.603     1.55           0
 3  UKVPN        0     4  RF    0.984    0.961   0.964  0.961  0.962  0.979      0  172.472     1.602          0
 4  UKVPN        0     5  RF    0.983    0.955   0.967  0.955  0.96   0.978      0  177.562     1.565          0
 5  UKVPN        1     1  RF    0.985    0.958   0.967  0.958  0.962  0.98       0  174.409     1.577          0
 6  UKVPN        1     2  RF    0.985    0.966   0.972  0.966  0.969  0.98       0  174.80

### Aggregated

In [32]:
savehere="results/cv/13/"
folder(savehere)
step=13
mixed=0
for file in file_list:
    print(file)
    train=file_list[file]
    test=file_list[file]
    output_csv=f"{savehere}{file}_ML_.csv"
    target_names=target_name(file_list[file])
    MLCV(train,test,output_csv,feature,step,mixed,file)   


UKVPN
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN        0     1  RF    1        1       1      1      1      1          0  169.525     1.651      0.858
 1  UKVPN        0     2  RF    1        1       1      1      1      1          0  172.888     1.576      0.83
 2  UKVPN        0     3  RF    1        1       1      1      1      1          0  171.822     1.511      0.843
 3  UKVPN        0     4  RF    1        1       1      1      1      1          0  171.832     1.52       0.833
 4  UKVPN        0     5  RF    1        1       1      1      1      1          0  174.943     1.503      0.858
 5  UKVPN        1     1  RF    1        1       1      1      1      1          0  172.991     1.644      0.849
 6  UKVPN        1     2  RF    1        1       1      1      1      1          0  181.107

# Session vs Session

In [16]:
file_list={"UKVPN@UK":['small/uk-vpn.csv','small/uk.csv'],
           "UK@UKVPN":['small/uk.csv','small/uk-vpn.csv'],
  "USVPN@US":['small/us-vpn.csv','small/us.csv'],
           "US@USVPN":['small/us.csv','small/us-vpn.csv']}

In [34]:
### Normal

In [35]:
say=0
savehere="results/ss/1/"
folder(savehere)


for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@UK
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@UK     0     1  RF    0.929    0.879   0.884  0.879  0.878  0.905      0  199.626     6.902          0
 1  UKVPN@UK     1     2  RF    0.928    0.879   0.884  0.879  0.879  0.904      0  197.78      6.933          0
 2  UKVPN@UK     2     3  RF    0.929    0.877   0.882  0.877  0.876  0.905      0  200.51      6.982          0
 3  UKVPN@UK     3     4  RF    0.931    0.878   0.884  0.878  0.878  0.907      0  203.453     7.663          0
 4  UKVPN@UK     4     5  RF    0.929    0.88    0.885  0.88   0.879  0.906      0  202.24      6.957          0
 5  UKVPN@UK     5     6  RF    0.929    0.879   0.885  0.879  0.879  0.905      0  201.617     6.956          0
 6  UKVPN@UK     6     7  RF    0.929    0.881   0.886  0.881  0.881  0.906      0  19

### Aggregated

In [17]:
say=0
savehere="results/ss/13/"
folder(savehere)


for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=13
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@UK
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@UK     0     1  RF    0.987    0.956   0.972  0.956  0.952  0.982      0  226.851    11.737      4.119
 1  UKVPN@UK     1     2  RF    0.986    0.952   0.965  0.952  0.944  0.981      0  242.591    12.454      3.251
 2  UKVPN@UK     2     3  RF    0.987    0.958   0.971  0.958  0.953  0.983      0  248.715    13.782      3.316
 3  UKVPN@UK     3     4  RF    0.988    0.961   0.976  0.961  0.955  0.984      0  237.859     7.136      3.853
 4  UKVPN@UK     4     5  RF    0.986    0.955   0.966  0.955  0.947  0.982      0  208.8       6.898      3.697
 5  UKVPN@UK     5     6  RF    0.986    0.956   0.971  0.956  0.951  0.982      0  207.741     7.09       3.791
 6  UKVPN@UK     6     7  RF    0.987    0.958   0.97   0.958  0.952  0.983      0  20

# Dataset Vs Dataset

In [18]:
file_list={"UKVPN@USVPN":['small/uk-vpn.csv','small/us-vpn.csv'],
"UKVPN@US":['small/uk-vpn.csv','small/us.csv'],
"UK@USVPN":['small/uk.csv','small/us-vpn.csv'],
"UK@US":['small/uk.csv','small/us.csv'],
"USVPN@UKVPN":['small/us-vpn.csv','small/uk-vpn.csv'],
"USVPN@UK":['small/us-vpn.csv','small/uk.csv'],
"US@UKVPN":['small/us.csv','small/uk-vpn.csv'],
"US@UK":['small/us.csv','small/uk.csv']}

### Normal

In [19]:
say=0
savehere="results/dd/1/"
folder(savehere)
for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@USVPN
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@USVPN    0     1  RF    0.833    0.8     0.784  0.8    0.767  0.794      0  202.378     7.67           0
 1  UKVPN@USVPN    1     2  RF    0.831    0.791   0.778  0.791  0.757  0.791      0  206.373     8.01           0
 2  UKVPN@USVPN    2     3  RF    0.83     0.771   0.779  0.771  0.738  0.79       0  201.939     7.824          0
 3  UKVPN@USVPN    3     4  RF    0.832    0.796   0.782  0.796  0.764  0.793      0  202.859     7.659          0
 4  UKVPN@USVPN    4     5  RF    0.828    0.767   0.771  0.767  0.729  0.788      0  206.048     7.544          0
 5  UKVPN@USVPN    5     6  RF    0.83     0.785   0.77   0.785  0.745  0.79       0  204.222     7.578          0
 6  UKVPN@USVPN    6     7  RF    0.828    0.776   0.776  0.776  0.

### Aggregated

In [20]:
say=0
savehere="results/dd/13/"
folder(savehere)
for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=13
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@USVPN
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@USVPN    0     1  RF    0.937    0.878   0.89   0.878  0.826  0.921      0  228.43      8.255      3.668
 1  UKVPN@USVPN    1     2  RF    0.937    0.873   0.89   0.873  0.824  0.921      0  205.828     7.699      3.728
 2  UKVPN@USVPN    2     3  RF    0.938    0.881   0.891  0.881  0.83   0.922      0  208.767     7.562      3.699
 3  UKVPN@USVPN    3     4  RF    0.94     0.894   0.891  0.894  0.846  0.925      0  206.622     7.496      3.653
 4  UKVPN@USVPN    4     5  RF    0.945    0.91    0.928  0.91   0.896  0.931      0  204.448     7.625      3.597
 5  UKVPN@USVPN    5     6  RF    0.945    0.91    0.928  0.91   0.895  0.93       0  207.118     7.551      3.574
 6  UKVPN@USVPN    6     7  RF    0.938    0.879   0.89   0.879  0.

In [19]:
200*25*2

10000

In [20]:
file_list={"USVPN":'small/us-vpn.csv',
           "US":'small/us.csv'}

In [21]:
savehere="results/cv/1/"
folder(savehere)
step=1
mixed=0
for file in file_list:
    print(file)
    train=file_list[file]
    test=file_list[file]
    output_csv=f"{savehere}{file}_ML_.csv"
    target_names=target_name(file_list[file])
    MLCV(train,test,output_csv,feature,step,mixed,file)   


USVPN
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  USVPN        0     1  RF    0.98     0.949   0.972  0.949  0.959  0.975      0  183.116     1.742          0
 1  USVPN        0     2  RF    0.979    0.96    0.97   0.96   0.965  0.973      0  177.47      1.555          0
 2  USVPN        0     3  RF    0.979    0.957   0.97   0.957  0.964  0.974      0  188.868     1.766          0
 3  USVPN        0     4  RF    0.978    0.962   0.97   0.962  0.966  0.973      0  203.933     1.81           0
 4  USVPN        0     5  RF    0.98     0.95    0.968  0.95   0.956  0.975      0  203.113     1.617          0
 5  USVPN        1     1  RF    0.979    0.956   0.971  0.956  0.963  0.974      0  190.383     1.588          0
 6  USVPN        1     2  RF    0.98     0.967   0.969  0.967  0.967  0.975      0  193.34

In [22]:
savehere="results/cv/13/"
folder(savehere)
step=13
mixed=0
for file in file_list:
    print(file)
    train=file_list[file]
    test=file_list[file]
    output_csv=f"{savehere}{file}_ML_.csv"
    target_names=target_name(file_list[file])
    MLCV(train,test,output_csv,feature,step,mixed,file)   


USVPN
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  USVPN        0     1  RF        1    0.96     0.96  0.96   0.96       1      0  168.466     1.463      0.661
 1  USVPN        0     2  RF        1    1        1     1      1          1      0  168.454     1.673      0.764
 2  USVPN        0     3  RF        1    0.997    1     0.997  0.999      1      0  172.238     1.449      0.793
 3  USVPN        0     4  RF        1    1        1     1      1          1      0  176.012     1.525      0.771
 4  USVPN        0     5  RF        1    0.997    1     0.997  0.998      1      0  170.681     1.49       0.752
 5  USVPN        1     1  RF        1    1        1     1      1          1      0  168.26      1.492      0.753
 6  USVPN        1     2  RF        1    0.997    1     0.997  0.999      1      0  171.88

In [23]:
!shutdown /s /t 360